# Trim Silence — Folder Batch

Trims leading and trailing silence from every WAV file in a folder.
Optionally filters out clips shorter than `MIN_DURATION` or longer than `MAX_DURATION`.

---

> No GPU required. Uses `librosa` + `soundfile`.

In [ ]:
import os

# ════════════════════════════════════════════
# ⚙️  CONFIGURATION
# ════════════════════════════════════════════
os.environ.setdefault("INPUT_DIR",      "/data/audio_in")
os.environ.setdefault("OUTPUT_DIR",     "/data/audio_trimmed")
os.environ.setdefault("TOP_DB",         "30")        # Silence threshold in dB
os.environ.setdefault("MIN_DURATION",   "0.5")       # Seconds — discard shorter clips
os.environ.setdefault("MAX_DURATION",   "30.0")      # Seconds — discard longer clips
os.environ.setdefault("TARGET_SR",      "")           # Resample after trim (empty = keep)

INPUT_DIR    = os.environ["INPUT_DIR"]
OUTPUT_DIR   = os.environ["OUTPUT_DIR"]
TOP_DB       = int(os.environ["TOP_DB"])
MIN_DUR      = float(os.environ["MIN_DURATION"])
MAX_DUR      = float(os.environ["MAX_DURATION"])
TARGET_SR    = os.environ["TARGET_SR"].strip()

print(f"Input:        {INPUT_DIR}")
print(f"Output:       {OUTPUT_DIR}")
print(f"Silence dB:   {TOP_DB}")
print(f"Duration:     {MIN_DUR}–{MAX_DUR} s")
print(f"Resample:     {TARGET_SR or 'keep original'}")

In [ ]:
!pip install --quiet --break-system-packages librosa soundfile tqdm

In [ ]:
import os
import librosa
import soundfile as sf
from pathlib import Path
from tqdm import tqdm

os.makedirs(OUTPUT_DIR, exist_ok=True)

files = sorted(Path(INPUT_DIR).rglob("*.wav"))
print(f"Found {len(files)} WAV files")

kept, too_short, too_long, skipped, failed = 0, 0, 0, 0, 0
for src in tqdm(files, desc="Trimming", unit="file"):
    rel = src.relative_to(INPUT_DIR)
    dst = Path(OUTPUT_DIR) / rel
    dst.parent.mkdir(parents=True, exist_ok=True)

    if dst.exists():
        skipped += 1
        continue

    try:
        sr_load = int(TARGET_SR) if TARGET_SR else None
        y, sr = librosa.load(str(src), sr=sr_load, mono=True)

        # Trim silence
        y_trimmed, _ = librosa.effects.trim(y, top_db=TOP_DB)

        dur = len(y_trimmed) / sr
        if dur < MIN_DUR:
            too_short += 1
            continue
        if dur > MAX_DUR:
            too_long += 1
            continue

        sf.write(str(dst), y_trimmed, sr)
        kept += 1
    except Exception as e:
        tqdm.write(f"  FAIL {src.name}: {e}")
        failed += 1

print(f"\nDone: {kept} kept, {too_short} too short, {too_long} too long, "
      f"{skipped} skipped, {failed} failed")